### Audio Event Classifier with Deep Learning

Build a CNN sound classifier using melspectograms from ESC-50 data. Refer to *save_melspectorgrams.ipynb* for feature extraction.

In [ ]:
%matplotlib inline
import keras
import tensorflow as tf
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.applications.vgg16 import preprocess_input
import numpy as np
import json

# Our version of the model ran on GPU training, this just serves to see if GPU 
# is detected by tensorflow
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

# Set parameter values

In [ ]:
batch_size = 40
epochs = 200

# dimensions of our images.
img_width, img_height = 224, 224

input_tensor = Input(shape=(224,224,3))

nb_training_samples = 1600
nb_validation_samples = 400# Set parameter values

# Configure training and validation data generators

Provide paths to training and testing set directores

In [ ]:
# training generator configuration
training_data_dir = 'melspectrograms/training'

training_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)

training_generator = training_datagen.flow_from_directory(
    training_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size)

# validation generator configuration
validation_data_dir ='melspectrograms/testing/'

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size)

# Make sure that you have the following results for the default execution:
# "Found 1600 images belonging to 50 classes."
# "Found 400 images belonging to 50 classes."

# Load base model

In [ ]:
# Loads VGG16 model pre-trained on ImageNet
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
print('Model loaded.')
base_model.summary()

# Build top model

In [ ]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(50, activation='softmax'))
top_model.summary()

# Combine base model with top model

In [ ]:
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))
model.summary()

#If you want to run the model with the weights from the previous run, uncomment the following line
#model.load_weights('esc50_weights.model.keras')


# Configure model training

In [ ]:
num_layers_to_freeze = 15

In [ ]:
from keras import metrics, optimizers

def top_5_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=5)

for layer in model.layers[:num_layers_to_freeze]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
model.compile(optimizer=optimizers.SGD(learning_rate=1e-4, momentum=0.9), 
                      loss='categorical_crossentropy', 
                      metrics=['accuracy', top_5_accuracy])

# serialize model to JSON
model_json = model.to_json()
model_filename = "vgg16_model_{}_frozen_layers.json".format(num_layers_to_freeze)
with open(model_filename, "w") as json_file:
    json_file.write(model_json)

# Fine-tune the model
# Do not run this cell if weights were loaded

In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard
from time import time

tensorboard = TensorBoard(log_dir="logs/layers_frozen_{}".format(num_layers_to_freeze))

# checkpoint
filepath="esc50_weights.model.keras"
best_model_checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [best_model_checkpoint, tensorboard]

# Fit the model

model.fit(
    training_generator,
    steps_per_epoch=int(nb_training_samples/batch_size),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=int(nb_validation_samples/batch_size),
    callbacks=callbacks_list)


# Get top k predictions for selected test files

In [ ]:
def get_top_k_predictions(preds, label_map, k=5, print_flag=False):
    sorted_array = np.argsort(preds)[::-1]
    top_k = sorted_array[:k]
    label_map_flip = dict((v,k) for k,v in label_map.items())
    
    y_pred = []
    for label_index in top_k:
        if print_flag:
            print("{} ({})").format(label_map_flip[label_index], preds[label_index])
        y_pred.append(label_map_flip[label_index])
        
    return y_pred

In [ ]:
label_map = (training_generator.class_indices)
map_json = json.dumps(label_map)
f = open("cough_label_map.json","w")
f.write(map_json)
f.close()

img_path = 'melspectrograms/testing/cat/1-34094-B-5.png'

img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)* 1./255

preds = model.predict(x)[0]

get_top_k_predictions(preds, label_map, k=3)

# Calculate and plot confusion matrix

In [ ]:
import itertools
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

#     print(cm)
    plt.figure(figsize=(24,24))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

In [ ]:
import os
from sklearn.metrics import confusion_matrix

testing_dir = 'melspectrograms/testing/'

y_true = []
y_pred = []
for label in label_map.keys():
    file_list = os.listdir(testing_dir + label)
    for file_name in file_list:
        img_path = testing_dir + label + '/' + file_name
        
        img = image.load_img(img_path, target_size=(224, 224))
        
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)* 1./255
        
        preds = model.predict(x)[0]
        
        y_true.append(label)
        y_pred.append(get_top_k_predictions(preds, label_map, k=1)[0])
        
cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, sorted(label_map.keys()), normalize=True)

In [ ]:
# Calculate accurate, precision and recall
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')

print("Accuracy: {}".format(accuracy))
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))